In [1]:
import pandas as pd

In [2]:

import tensorflow as tf
tf.config.list_physical_devices('GPU')


[]

In [3]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [4]:


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17893514372999188996
xla_global_id: -1
]


In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]